In [119]:
# Import libraries

from pyserini.search import pysearch
import pandas as pd
import json
from transformers import BertTokenizer, BartTokenizer, BartForConditionalGeneration, BertForQuestionAnswering
import torch
from tqdm import tqdm
import numpy as np

In [120]:
query = 'How does weather, heat, and humidity affect the tramsmission of 2019-nCoV'
keywords = ''
hit_dict = {}

searcher = pysearch.SimpleSearcher('lucene-index/')
hits = searcher.search(query + '. ' + keywords, 10)

In [121]:
#Exlporatory Pandas Stuff

#ranks = range(1, len(hits)+1)
#docids = [hit.docid for hit in hits]
#scores = [hit.score for hit in hits]
#titles = [ hit.lucene_document.get('title') for hit in hits]
#dois = [ hit.lucene_document.get('doi') for hit in hits]
#data = {'rank': ranks, 'docid': docids, 'score': scores, 'title': titles, 'doi': dois} 
#
#df = pd.DataFrame(data)
#print(df)'''

In [122]:
# Populate hit_dict from Lucene index

for hit in hits:
    key = hit.docid[:8]
    json_result = searcher.doc(key).raw()
    if len(json_result):
        article = json.loads(searcher.doc(key).raw())
        hit_dict[key] = {}
        hit_dict[key]['title'] = article['metadata']['title']
        hit_dict[key]['authors'] = article['metadata']['authors']
        hit_dict[key]['doi'] = hit.lucene_document.get('doi')
        hit_dict[key]['score'] = hit.score
        hit_dict[key]['body'] = article['body_text']
        if 'abstract' in article.keys():
            if len(article['abstract']):
                hit_dict[key]['abstract'] = article['abstract'][0]['text']
        print(hit_dict[key].keys())

# Each entry in hit_dict may or may not have an abstract

dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])


In [123]:
# Print top search results

i = 1
for key in hit_dict.keys():
    print(f"{i}.\n\
    \t- Score: {hit_dict[key]['score']}\n\
    \t- Title: {hit_dict[key]['title']}\n\
    \t- Abstract: {hit_dict[key]['abstract'][:150] + '...' if 'abstract' in hit_dict[key].keys() else 'N/A'}\n\
    \t- Body: {hit_dict[key]['body'][0]['text'][:200]}...\n\
    \t- DOI: {hit_dict[key]['doi']}\n\
    \t- Authors: {[author['first'] + ' ' + author['last'] for author in hit_dict[key]['authors']]}\n")
    i += 1

1.
    	- Score: 13.340998649597168
    	- Title: Roles of meteorological conditions in COVID-19 transmission on a worldwide scale
    	- Abstract: The novel coronavirus (SARS-CoV-2/ 2019-nCoV) identified in Wuhan, China, in December 2019 has caused great damage to public health and economy worldw...
    	- Body: In the first season of 2020, an outbreak of atypical pneumonia (COVID-19) caused by a novel coronavirus (SARS-CoV2 or 2019-nCoV) 1 has spread all over the world and had a great impact on public health...
    	- DOI: 10.1101/2020.03.16.20037168
    	- Authors: ['Biqing Chen', 'Hao Liang', 'Xiaomin Yuan', 'Yingying Hu', 'Miao Xu', 'Yating Zhao', 'Binfen Zhang', 'Xuejun Zhu']

2.
    	- Score: 14.552000045776367
    	- Title: Journal Pre-proof Investigation of effective climatology parameters on COVID-19 outbreak in Iran Investigation of Effective Climatology Parameters on COVID-19 Outbreak in Iran
    	- Abstract: SARS CoV-2 (COVID-19) Coronavirus cases are confirmed throughout 

In [124]:
# Organization of hit_dict

key = ''
for i in hit_dict.keys():
    if 'abstract' in hit_dict[i].keys():
        key = i
        break

print(f'hit_dict[key]:\n\t{type(hit_dict[key])}\n\t{hit_dict[key].keys()}')

print('===')
print(f'\nhit_dict[key]["title"]:\n\t{type(hit_dict[key]["title"])}')

print('===')
print(f'\nhit_dict[key]["authors"]:\n\t{type(hit_dict[key]["authors"])}')
print(f'\n\thit_dict[key]["authors"][author_index]:\n\t\t{type(hit_dict[key]["authors"][0])}\n\t\t{hit_dict[key]["authors"][0].keys()}')

print('===')
print(f'\nhit_dict[key]["doi"]:\n\t{type(hit_dict[key]["doi"])}')

print('===')
print(f'\nhit_dict[key]["score"]:\n\t{type(hit_dict[key]["score"])}')

print('===')
print(f'\nhit_dict[key]["body"]:\n\t{type(hit_dict[key]["body"])}')
print(f'\n\thit_dict[key]["body"][paragraph_index]:\n\t\t{type(hit_dict[key]["body"][0])}\n\t\t{hit_dict[key]["body"][0].keys()}')

print('===')
print(f'\nhit_dict[key]["abstract"]:\n\t{type(hit_dict[key]["abstract"])}')


hit_dict[key]:
	<class 'dict'>
	dict_keys(['title', 'authors', 'doi', 'score', 'body', 'abstract'])
===

hit_dict[key]["title"]:
	<class 'str'>
===

hit_dict[key]["authors"]:
	<class 'list'>

	hit_dict[key]["authors"][author_index]:
		<class 'dict'>
		dict_keys(['first', 'middle', 'last', 'suffix', 'affiliation', 'email'])
===

hit_dict[key]["doi"]:
	<class 'str'>
===

hit_dict[key]["score"]:
	<class 'float'>
===

hit_dict[key]["body"]:
	<class 'list'>

	hit_dict[key]["body"][paragraph_index]:
		<class 'dict'>
		dict_keys(['text', 'cite_spans', 'ref_spans', 'section'])
===

hit_dict[key]["abstract"]:
	<class 'str'>


In [125]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_MODEL.to(torch_device)
QA_MODEL.eval()

SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('bart-large-cnn')
SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
SUMMARY_MODEL.to(torch_device)
SUMMARY_MODEL.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): SelfAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tr

In [126]:
def makeBERTSQuADPrediction(document, question):
    nWords = len(document.split())
    input_ids_all = QA_TOKENIZER.encode(question, document)
    tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
    overlapFac = 1.1
    if len(input_ids_all)*overlapFac > 2048:
        nSearchWords = int(np.ceil(nWords/5))
        quarter = int(np.ceil(nWords/4))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        
    elif len(input_ids_all)*overlapFac > 1536:
        nSearchWords = int(np.ceil(nWords/4))
        third = int(np.ceil(nWords/3))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        
    elif len(input_ids_all)*overlapFac > 1024:
        nSearchWords = int(np.ceil(nWords/3))
        middle = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    elif len(input_ids_all)*overlapFac > 512:
        nSearchWords = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    else:
        input_ids = [input_ids_all]
    absTooLong = False    
    
    answers = []
    cons = []
    for iptIds in input_ids:
        tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
        sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(iptIds) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        assert len(segment_ids) == len(iptIds)
        n_ids = len(segment_ids)

        if n_ids < 512:
            start_scores, end_scores = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids]).to(torch_device))
        else:
            absTooLong = True
            start_scores, end_scores = QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device),
                                     token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
        start_scores = start_scores[:,1:-1]
        end_scores = end_scores[:,1:-1]
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        answer = reconstructText(tokens, answer_start, answer_end+2)
    
        if answer.startswith('. ') or answer.startswith(', '):
            answer = answer[2:]
            
        c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
        answers.append(answer)
        cons.append(c)
    
    maxC = max(cons)
    iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
    confidence = cons[iMaxC]
    answer = answers[iMaxC]
    
    sep_index = tokens_all.index('[SEP]')
    full_txt_tokens = tokens_all[sep_index+1:]
    
    abs_returned = reconstructText(full_txt_tokens)

    ans={}
    ans['answer'] = answer
    if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
        ans['confidence'] = -1000000
    else:
        ans['confidence'] = confidence
    ans['abstract_bert'] = abs_returned
    ans['abs_too_long'] = absTooLong
    return ans

In [127]:
def reconstructText(tokens, start=0, stop=-1):
    tokens = tokens[start: stop]
    if '[SEP]' in tokens:
        sepind = tokens.index('[SEP]')
        tokens = tokens[sepind+1:]
    txt = ' '.join(tokens)
    txt = txt.replace(' ##', '')
    txt = txt.replace('##', '')
    txt = txt.strip()
    txt = " ".join(txt.split())
    txt = txt.replace(' .', '.')
    txt = txt.replace('( ', '(')
    txt = txt.replace(' )', ')')
    txt = txt.replace(' - ', '-')
    txt_list = txt.split(' , ')
    txt = ''
    nTxtL = len(txt_list)
    if nTxtL == 1:
        return txt_list[0]
    newList =[]
    for i,t in enumerate(txt_list):
        if i < nTxtL -1:
            if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                newList += [t,',']
            else:
                newList += [t, ', ']
        else:
            newList += [t]
    return ''.join(newList)

In [128]:
def searchAbstracts(hit_dictionary, question):
    abstractResults = {}
    for k,v in tqdm(hit_dictionary.items()):
        if 'abstract' in v.keys():
            abstract = v['abstract']
        else:
            abstract = None
        if abstract:
            ans = makeBERTSQuADPrediction(abstract, question)
            if ans['answer']:
                confidence = ans['confidence']
                abstractResults[confidence]={}
                abstractResults[confidence]['answer'] = ans['answer']
                abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                abstractResults[confidence]['idx'] = k
                abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']
                
    cList = list(abstractResults.keys())

    if cList:
        maxScore = max(cList)
        total = 0.0
        exp_scores = []
        for c in cList:
            s = np.exp(c-maxScore)
            exp_scores.append(s)
        total = sum(exp_scores)
        for i,c in enumerate(cList):
            abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
    return abstractResults

In [129]:
abstractResults = searchAbstracts(hit_dict, query)

100%|██████████| 3/3 [00:02<00:00,  1.07it/s]


In [130]:
print(abstractResults[list(abstractResults.keys())[0]]['abstract_bert'])

the novel coronavirus (sars-cov-2 / 2019-ncov) identified in wuhan, china, in december 2019 has caused great damage to public health and economy worldwide with over 140,000 infected cases up to date. previous research has suggested an involvement of meteorological conditions in the spread of droplet-mediated viral diseases, such as influenza. however, as for the recent novel coronavirus, few studies have discussed systematically about the role of daily weather in the epidemic transmission of the virus. here, we examine the relationships of meteorological variables with the severity of the outbreak on a worldwide scale. the confirmed case counts, which indicates the severity of covid-19 spread, and four meteorological variables, i. e., air temperature, relative humidity, wind speed, and visibility, were collected daily between january 20 and march 11 (52 days) for 430 cities and districts. cc-by-nc-nd 4. 0 international license it is made available under a is the author / funder, who ha